In [22]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from sklearn import model_selection, tree

In [3]:
HTML_markup = (requests.get("https://fbref.com/en/squads/361ca564/Tottenham-Hotspur-Stats")).text
parsed_code = BeautifulSoup(HTML_markup, 'html.parser')

In [4]:
parsed_code


<!DOCTYPE html>

<html class="no-js" data-root="/home/fb/deploy/www/base" data-version="klecko-" lang="en">
<head>
<meta charset="utf-8"/>
<meta content="ie=edge" http-equiv="x-ua-compatible"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=2.0" name="viewport">
<link href="https://cdn.ssref.net/req/202309261" rel="dns-prefetch"/>
<!-- Quantcast Choice. Consent Manager Tag v2.0 (for TCF 2.0) -->
<script async="true" type="text/javascript">
    (function() {
	var host = window.location.hostname;
	var element = document.createElement('script');
	var firstScript = document.getElementsByTagName('script')[0];
	var url = 'https://cmp.quantcast.com'
	    .concat('/choice/', 'XwNYEpNeFfhfr', '/', host, 
		    '/choice.js?tag_version=V2');
	var uspTries = 0;
	var uspTriesLimit = 3;
	element.async = true;
	element.type = 'text/javascript';
	element.src = url;
	
	firstScript.parentNode.insertBefore(element, firstScript);
	
	function makeStub() {
	    var TCF_LOCATOR_NAME = '

In [5]:
def extract_info(parsed_code):
    date_elements = parsed_code.find_all("th", {"data-stat": "date"})
    dates = []
    
    goals_for_elements = parsed_code.find_all("td", {"data-stat": "goals_for"})
    goals_for_values = []
    
    goals_against_elements = parsed_code.find_all("td", {"data-stat": "goals_against"})
    goals_against_values = []
    
    opponent_elements = parsed_code.find_all("td", {"data-stat": "opponent"})
    opponents = []
    
    venue_elements = parsed_code.find_all("td", {"data-stat":"venue"})
    venues = []
    
    for date_element, goals_for_element, goals_against_element, opponent_element, venue_element in zip(date_elements, goals_for_elements, goals_against_elements, opponent_elements, venue_elements):
        date = date_element.a.get_text() if date_element.a else date_element.get_text()
        goals_for = goals_for_element.get_text()
        goals_against = goals_against_element.get_text()
        opponent = opponent_element.a.get_text() if opponent_element.a else opponent_element.get_text()
        venue = venue_element.get_text()
        
        if date and goals_for and goals_against and opponent and venue:
            dates.append(date)
            goals_for_values.append(goals_for)
            goals_against_values.append(goals_against)
            opponents.append(opponent)
            venues.append(venue)
    return dates, goals_for_values, goals_against_values, opponents, venues


In [6]:
years = []

for year in range(2023, 2000, -1):
    academic_year = f"{year}-{year + 1}"
    years.append(academic_year)

# Print the list of academic years
print(years)

['2023-2024', '2022-2023', '2021-2022', '2020-2021', '2019-2020', '2018-2019', '2017-2018', '2016-2017', '2015-2016', '2014-2015', '2013-2014', '2012-2013', '2011-2012', '2010-2011', '2009-2010', '2008-2009', '2007-2008', '2006-2007', '2005-2006', '2004-2005', '2003-2004', '2002-2003', '2001-2002']


In [7]:
dates = []
goals_for = []
goals_against = []
opponents = []
venues = []
for year in years:
    HTML_markup = requests.get(f"https://fbref.com/en/squads/361ca564/{year}/Tottenham-Hotspur-Stats").text
    parsed_code = BeautifulSoup(HTML_markup, 'html.parser')
    
    extracted_data = extract_info(parsed_code)
    
    dates.extend(extracted_data[0])
    goals_for.extend(extracted_data[1])
    goals_against.extend(extracted_data[2])
    opponents.extend(extracted_data[3])
    venues.extend(extracted_data[4])

In [8]:
print(len(goals_for))
print(len(goals_against))
print(len(opponents))
print(len(dates))
print(len(venues))

1068
1068
1068
1068
1068


In [9]:
dates = dates[1:]
dates.append('2002-05-11')

In [11]:
import pandas as pd

data_values = ["Date", "Goals_For", "Goals_Against", "Opponents", "Venue"]
data_dict = {
    "Date": dates,
    "Goals_For": goals_for,
    "Goals_Against": goals_against,
    "Opponents": opponents,
    "Venue":venues
}

finished_match_data = pd.DataFrame(data_dict)

In [12]:
def calculate_match_result(row):
    goals_for = row['Goals_For']
    goals_against = row['Goals_Against']

    # Check if goals for > goals against
    if goals_for > goals_against:
        return 2
    # Check if the scores are the same and compare the scores in brackets
    elif goals_for == goals_against:
        goals_for_in_brackets = goals_against_in_brackets = 0
        if '(' in str(goals_for):
            goals_for_in_brackets = int(goals_for.split(' ')[1].strip('()'))
        if '(' in str(goals_against):
            goals_against_in_brackets = int(goals_against.split(' ')[1].strip('()'))
        
        if goals_for_in_brackets > goals_against_in_brackets:
            return 2
        else:
            return 1
    return 0

# Apply the function to create the 'Match Result' column
finished_match_data['Match Result'] = finished_match_data.apply(calculate_match_result, axis=1)

In [13]:
finished_match_data

,Date,Goals_For,Goals_Against,Opponents,Venue,Match Result
0,2023-08-13,2,2,Brentford,Away,1
1,2023-08-19,2,0,Manchester Utd,Home,2
2,2023-08-26,2,0,Bournemouth,Away,2
3,2023-08-29,1 (3),1 (5),Fulham,Away,0
4,2023-09-02,5,2,Burnley,Away,2
...,...,...,...,...,...,...
1063,2002-04-06,1,2,Arsenal,Away,0
1064,2002-04-13,1,1,West Ham,Home,1
1065,2002-04-20,1,1,Bolton,Away,1
1066,2002-04-27,1,0,Liverpool,Home,2


In [29]:
train_data, test_data = model_selection.train_test_split(finished_match_data)
train_data_output = train_data["Match Result"]
train_data.drop(columns=["Match Result"], inplace=True)
test_data_output = test_data["Match Result"]
test_data.drop(columns=["Match Result"], inplace=True)

ValueError: could not convert string to float: '2019-12-26'